# 📍 Star Catalog Triangulation Tutorial

This notebook demonstrates how a spacecraft can **triangulate its orientation** (attitude) using a star catalog and measurements from a star tracker.

We'll simplify to a 2D sky view and use synthetic star directions to show how attitude can be solved by matching observed directions with a known catalog.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R

# Create a synthetic 'catalog' of star directions (unit vectors in inertial frame)
catalog = np.array([
    [1, 0],
    [0.5, np.sqrt(3)/2],
    [-0.5, np.sqrt(3)/2],
    [-1, 0],
    [0, -1]
])
catalog = catalog / np.linalg.norm(catalog, axis=1, keepdims=True)

### 🎯 Define a spacecraft orientation (rotation)
We'll rotate the frame by 30 degrees to simulate the star tracker measurement.

In [ ]:
# Define a spacecraft rotation (e.g., 30 degrees counterclockwise)
theta_deg = 30
rotation = R.from_euler('z', theta_deg, degrees=True)
observed = rotation.apply(catalog)

### 🧭 Solve for the orientation using Wahba’s Problem (Kabsch Algorithm)
We solve for the rotation that best aligns the observed vectors to catalog vectors.
This is a basic approach used by star trackers to compute attitude.

In [ ]:
def solve_attitude(observed, catalog):
    H = observed.T @ catalog
    U, _, Vt = np.linalg.svd(H)
    R_est = Vt.T @ U.T
    return R.from_matrix(R_est)

# Estimate rotation
R_est = solve_attitude(observed, catalog)
estimated_angle = R_est.as_euler('zxy', degrees=True)[0]
print(f"Estimated rotation angle: {estimated_angle:.2f} degrees")

### 🔭 Visualize the original vs observed vs aligned stars

In [ ]:
aligned = R_est.apply(observed)

plt.figure(figsize=(6, 6))
plt.quiver(np.zeros(5), np.zeros(5), catalog[:,0], catalog[:,1], angles='xy', scale_units='xy', scale=1, color='blue', label='Catalog')
plt.quiver(np.zeros(5), np.zeros(5), observed[:,0], observed[:,1], angles='xy', scale_units='xy', scale=1, color='red', alpha=0.6, label='Observed')
plt.quiver(np.zeros(5), np.zeros(5), aligned[:,0], aligned[:,1], angles='xy', scale_units='xy', scale=1, color='green', alpha=0.6, label='Aligned')

plt.legend()
plt.grid(True)
plt.axis('equal')
plt.title("Star Vectors: Catalog vs Observed vs Aligned")
plt.show()

## ✅ Summary
- Star trackers measure directions to stars.
- A known catalog allows matching to find spacecraft orientation.
- We demonstrated this using synthetic stars and Wahba’s problem.

This 2D example mirrors real 3D star tracker techniques like QUEST or TRIAD used in spacecraft flight software.